Elaborado por Lissette Monserrat Garza Rivas & Raul Solis Molinar

<a id = "Imports"></a>
# Importar bibliotecas y configurar algunas funciones auxiliares
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
# Librerias Basicas
import os
import numpy as np
import pandas as pd

# esto nos permitirá imprimir todos los archivos a medida que generamos más en el kernel
def print_files():
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            print(os.path.join(dirname, filename))

# consultar el truco 91 para ver un ejemplo
def generate_sample_data(): # creates a fake df for testing
    number_or_rows = 20
    num_cols = 7
    cols = list("ABCDEFG")
    df = pd.DataFrame(np.random.randint(1, 20, size = (number_or_rows, num_cols)), columns=cols)
    df.index = pd.util.testing.makeIntIndex(number_or_rows)
    return df

# consultar el truco 91 por el ejemplo
def generate_sample_data_datetime(): # crea un df falso para probar
    number_or_rows = 365*24
    num_cols = 2
    cols = ["sales", "customers"]
    df = pd.DataFrame(np.random.randint(1, 20, size = (number_or_rows, num_cols)), columns=cols)
    df.index = pd.util.testing.makeDateIndex(number_or_rows, freq="H")
    return df

# mostrar varias impresiones en una celda. Esto nos permitirá condensar cada truco en una celda.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

print_files()

<a id = "trick100"></a>
# Trick 100: Cargando muestra de big data
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = pd.read_csv("/kaggle/input/us-accidents/US_Accidents_Dec19.csv")
print("The shape of the df is {}".format(df.shape))

del df

df = pd.read_csv("/kaggle/input/us-accidents/US_Accidents_Dec19.csv", skiprows = lambda x: x>0 and np.random.rand() > 0.01)
print("The shape of the df is {}. It has been reduced 10 times!".format(df.shape))


'''
Cómo funciona:
skiprows acepta una función que se evalúa con el índice entero.
x> 0 se asegura de que los encabezados no se omitan
np.random.rand ()> 0.01 devuelve True 99% del empate, por lo que se salta el 99% del tiempo.
Tenga en cuenta que estamos usando skiprows
'''

<a id = "trick99"></a>
# Trick 99: Cómo evitar Innominado: 0 columnas
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
d = {\
"zip_code": [12345, 56789, 101112, 131415],
"factory": [100, 400, 500, 600],
"warehouse": [200, 300, 400, 500],
"retail": [1, 2, 3, 4]
}

df = pd.DataFrame(d)
df

# guadar en csv
df.to_csv("trick99data.csv")

df = pd.read_csv("trick99data.csv")
df
# Para evitar Innominado: 0

df = pd.read_csv("trick99data.csv", index_col=0)
# Cuando guardes df = pd.read_csv("trick99data.csv", index = False)
df

<a id = "trick98"></a>
# Trick 98: Convertir un DF ancho en uno largo
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
d = {\
"zip_code": [12345, 56789, 101112, 131415],
"factory": [100, 400, 500, 600],
"warehouse": [200, 300, 400, 500],
"retail": [1, 2, 3, 4]
}

df = pd.DataFrame(d)
df

# Nosotros tenemos que asignar

# location_type se genera automáticamente a partir de las columnas que quedan después especificando id_vars (también puedes pasar una lista)
df = df.melt(id_vars = "zip_code", var_name = "location_type", value_name = "distance")
df

<a id = "trick97"></a>
# Trick 97: Convertir año y día del año en una sola columna de fecha y hora
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
# Trick 97
# Convert
d = {\
"year": [2019, 2019, 2020],
"day_of_year": [350, 365, 1]
}

df = pd.DataFrame(d)
df

# Step 1: crear una columna combinada
df["combined"] = df["year"]*1000 + df["day_of_year"]
df

# Step 2: convertir a fecha y hora
df["date"] = pd.to_datetime(df["combined"], format = "%Y%j")
df

<a id = "trick96"></a>
# Trick 96: Parcelas interactivas listas para usar en pandas
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
print(pd.__version__)
# Se requiere la versión 0.25 o superior de Pandas y necesita hvplot

import pandas as pd
df = pd.read_csv("../input/drinks-by-country/drinksbycountry.csv")
df

# este no es interactivo
df.plot(kind = "scatter", x = "spirit_servings", y = "wine_servings")

# run !pip install hvplot
#pd.options.plotting.backend = "hvplot"
#df.plot(kind = "scatter", x = "spirit_servings", y = "wine_servings", c = "continent")

<a id = "trick95"></a>
# Trick 95: Contar los valores perdidos
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
d = {\
"col1": [2019, 2019, 2020],
"col2": [350, 365, 1],
"col3": [np.nan, 365, None]
}

df = pd.DataFrame(d)
df

# Solución 1
df.isnull().sum().sum()

# Solución 2
df.isna().sum()

# Solución 3
df.isna().any()

# Solución 4:
df.isna().any(axis = None)

<a id = "trick94"></a>
# Trick 94: Ahorrar memoria fijando su fecha
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = pd.read_csv("../input/titanic/train.csv", usecols = ["Pclass", "Sex", "Parch", "Cabin"])
df

# veamos cuanto ocupa nuestro df en memoria
df.memory_usage(deep = True)

# convertir a tipos de datos más pequeños
df = df.astype({"Pclass":"int8",
                "Sex":"category", 
                "Parch": "Sparse[int]", # la mayoría de los valores son 0
                "Cabin":"Sparse[str]"}) # la mayoría de los valores son NaN

df.memory_usage(deep = True)

<a id = "trick93"></a>
# Trick 93: Combinar las categorías pequeñas en una sola categoría llamada "Otros" (usando frecuencias)
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
d = {"genre": ["A", "A", "A", "A", "A", "B", "B", "C", "D", "E", "F"]}
df = pd.DataFrame(d)
df

# Step 1: contar las frecuencias
frequencies = df["genre"].value_counts(normalize = True)
frequencies

# Step 2: establezca su umbral y filtre las categorías más pequeñas
threshold = 0.1
small_categories = frequencies[frequencies < threshold].index
small_categories

# Step 3: reemplazar los valores
df["genre"] = df["genre"].replace(small_categories, "Other")
df["genre"].value_counts(normalize = True)

<a id = "trick92"></a>
# Trick 92: Limpiar la columna de objetos con datos mixtos usando expresiones regulares
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
d = {"customer": ["A", "B", "C", "D"], "sales":[1100, 950.75, "$400", "$1250.35"]}
df = pd.DataFrame(d)
df

# Step 1: comprobar los tipos de datos
df["sales"].apply(type)

# Step 2: usar regex
df["sales"] = df["sales"].replace("[$,]", "", regex = True).astype("float")
df
df["sales"].apply(type)

<a id = "trick91"></a>
# Trick 91: Crear un conjunto de datos de series de tiempo para realizar pruebas
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
# Solución 1
number_or_rows = 365*24 # horas en el año
pd.util.testing.makeTimeDataFrame(number_or_rows, freq="H")

# Solución 2
num_cols = 2
cols = ["sales", "customers"]
df = pd.DataFrame(np.random.randint(1, 20, size = (number_or_rows, num_cols)), columns=cols)
df.index = pd.util.testing.makeDateIndex(number_or_rows, freq="H")
df

<a id = "trick90"></a>
# Trick 90: Mover columnas a una ubicación específica
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
d = {"A":[15, 20], "B":[20, 25], "C":[30 ,40], "D":[50, 60]}
df = pd.DataFrame(d)
df

# Usando insertar
df.insert(3, "C2", df["C"]*2)
df

# otra solución
df["C3"] = df["C"]*3 # crea nuevas columnas, estará al final
columns = df.columns.to_list() # crea una lista con todas las columnas
location = 4 # especifique la ubicación donde desea su nueva columna
columns = columns[:location] + ["C3"] + columns[location:-1] # reaorganizar la lista
df = df[columns] # cree el marco de datos con el orden de columnas que desee
df


<a id = "trick89"></a>
# Trick 89: Dividir nombres en nombre y apellido
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = pd.Series(["Geordi La Forge", "Deanna Troi", "Data"]).to_frame()
df.rename({0:"names"}, inplace = True, axis = 1)
df
#   dividir en el primer espacio 
df["first_name"] = df["names"].str.split(n = 1).str[0]
df["last_name"] = df["names"].str.split(n = 1).str[1]
df

<a id = "trick88"></a>
# Trick 88: Reacomodar columnas en df
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = generate_sample_data()
df.head()

# Solución 1
df[["A", "C", "D", "F", "E", "G", "B"]].head() # no se modifica en su lugar

# Solución 2
cols_to_move = ["A", "G", "B"]

new_order = cols_to_move + [c for c in df.columns if c not in cols_to_move] # genera un nuevo orden
df[new_order].head()

# Solución 3: 
cols = df.columns[[0, 5 , 3, 4, 2, 1, 6]] # df.columns devuelve una serie con índice, usamos la lista para ordenar el índice como queramos -> de esta manera ordenamos las columnas
df[cols].head()


<a id = "trick87"></a>
# Trick 87: Agregar su fecha y hora y filtrar los fines de semana
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = generate_sample_data_datetime()
df.shape
df.head()

# Step 1:remuestrear por D. Básicamente agregue por día y use to_frame () para convertirlo en marco
daily_sales = df.resample("D")["sales"].sum().to_frame()
daily_sales

# Step 2: filtrar el fin de semana
weekends_sales = daily_sales[daily_sales.index.dayofweek.isin([5, 6])]
weekends_sales

'''
día de la semana
0 lunes
1 martes
2 miércoles
3 jueves
4 viernes
5 sábado
6 domingo
'''

<a id = "trick86"></a>
# Trick 86: Agregaciones con nombre: evita el índice múltiple
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
print_files()

df = pd.read_csv("/kaggle/input/titanic/train.csv")
df.head()

# Problema 1
print("The Problem relies on that we don't know the column name")
df.groupby("Pclass")["Age"].agg(["mean", "max"])

# Problema 2
print("The Problem relies on that we have multiindex")
df.groupby("Pclass").agg({"Age":["mean", "max"]})

# Solución nueva en pandas 0.25 y superior
print("Now we have solved the previous problems by specifyig the column final name we want.")
print("BUT IT ONLY WORKS WITH A COLUMN. TO THIS KIND OF OPERATIONS ON MULTIPLE COLUMNS CHECK THE NEXT CELL")
df.groupby("Pclass")["Age"].agg(age_mean = "mean", age_max = "max")


<a id = "trick86bis"></a>
# Trick 86bis: Agregaciones con nombre en varias columnas: evita el índice múltiple
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
def my_agg(x):
    names = {
        'age_mean': x['Age'].mean(),
        'age_max':  x['Age'].max(),
        'fare_mean': x['Fare'].mean(),
        'fare_max': x['Fare'].max()
    } # definir sus operaciones y nombres de columnas personalizados

    return pd.Series(names, index=[ key for key in names.keys()]) # todas las columnas que cree en el diccionario anterior estarán en esta lista de comprensión

df.groupby('Pclass').apply(my_agg)

# referencia
# https://stackoverflow.com/questions/44635626/rename-result-columns-from-pandas-aggregation-futurewarning-using-a-dict-with


<a id = "trick85"></a>
# Trick 85: Convertir un tipo de valores en otros
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
# Haz algunas funciones rápidas en el DF
d = {"gender":["male", "female", "male"], "color":["red", "green", "blue"], "age":[25, 30, 15]}
df = pd.DataFrame(d)
df

# Solución
map_dict = {"male":"M", "female":"F"}
df["gender_mapped"] = df["gender"].map(map_dict) # usar diccionarios para mapear valores
df["color_factorized"] = df["color"].factorize()[0] # using factorize: returns a tuple of arrays (array([0, 1, 2]), Index(['red', 'green', 'blue'], dtype='object')) that's why we select [0]
df["age_compared_boolean"] = df["age"] < 18 # devuelve un valor booleano True False

df

<a id = "trick84"></a>
# Trick 84: Mostrar menos filas en un df
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
print("This df occupies way too much space")
df = generate_sample_data()
df

print("using set_option to save some screen space")
pd.set_option("display.max_rows", 6)
df

print("use reset_option all to reset to default")
pd.reset_option("all")
df

<a id = "trick83"></a>
# Trick 83: Corregir los tipos de datos al importar el df
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
print_files()
df = pd.read_csv("/kaggle/input/drinks-by-country/drinksbycountry.csv")

# Step 1: Vamos al tipo de fecha de las columnas
col_types = df.dtypes.to_frame()
col_types.rename({0:"type"}, inplace = True, axis = 1)
col_types
col_types.to_csv("trick83data.csv")

# Step 2: Vamos a importar los datos anteriores y convertirlos a un diccionario.
col_dict = pd.read_csv("trick83data.csv", index_col = 0)["type"].to_dict()

# Step 3: Edite el diccionario con los tipos de datos correctos
print("Original dictionary")
col_dict
col_dict["country"] = "category"
col_dict["continent"] = "category"
print("Modified dictionary")
col_dict

# Step 4: Usa el diccionario para importar los datos
df = pd.read_csv("/kaggle/input/drinks-by-country/drinksbycountry.csv", dtype=col_dict)
df.dtypes

# Note: tenga en cuenta que puede usar el dict del paso 1 y pegarlo así
df = pd.read_csv("/kaggle/input/drinks-by-country/drinksbycountry.csv", \
dtype=
{'country': 'category',
 'beer_servings': 'int64',
 'spirit_servings': 'int64',
 'wine_servings': 'int64',
 'total_litres_of_pure_alcohol': 'float64',
 'continent': 'category'})
# Sin embargo, si tiene muchas columnas, esto puede resultar confuso.
df.dtypes

<a id = "trick82"></a>
# Trick 82: Seleccionar datos por etiqueta y posición (chained iloc and loc)
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = pd.read_csv("/kaggle/input/drinks-by-country/drinksbycountry.csv", index_col="country")
df.iloc[15:20, :].loc[:, "beer_servings":"wine_servings"]
# iloc se usa para filtrar las filas y ubicar las columnas

<a id = "trick81"></a>
# Trick 81: Usar aplicar (type) para ver si tiene tipos de datos mixtos
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
d = {"customer":["A", "B", "C", "D", "E"], "sales":[100, "100", 50, 550.20, "375.25"]}
df = pd.DataFrame(d)
#todo parece pero esta operación falla df["sales"].sum(). Tenemos tipos de datos mixtos
df.dtypes
df["sales"].apply(type) # Wow, podemos ver que tenemos int, str, flota en una columna
df["sales"].apply(type).value_counts() # Ver el número de cada valor

df["sales"] = df["sales"].astype(float) # convertir los datos a flotar
df["sales"].sum()
df["sales"].apply(type).value_counts()

<a id = "trick80"></a>
# Trick 80: Seleccione varios sectores de columnas de un df
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = generate_sample_data().T
cols_str = list(map(str, list(df.columns))) # para que podamos hacer df ["0"] como cadena para el ejemplo
df.columns = cols_str

# Usando la concatenación de pandas
# si alguna vez está confundido acerca del eje = 1 o el eje = 0, simplemente coloque eje = "columnas" o eje = "filas"
pd.concat([df.loc[:, "0":"2"], df.loc[:, "6":"10"], df.loc[:, "16":"19"]], axis = "columns") # ------------------> aquí estamos seleccionando columnas convertidas en cadenas

# Usando listas
# tenga en cuenta que df.columns es una serie con índice, por lo que estamos usando índice para filtrar # -------------------------> aquí estamos seleccionando el índice de columnas
df[list(df.columns[0:3]) + list(df.columns[6:11]) + list(df.columns[16:20])]

# Usando numpy
df.iloc[:, np.r_[0:3, 6:11, 16:20]] # probablemente la solución más hermosa

<a id = "trick79"></a>
# Trick 79: Recuento de filas que coinciden con una condición
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = generate_sample_data()
df.head()
df.shape

# Valores absolutos
(df["A"] < 5).sum()
print("In the columns A we have {} of rows that are below 5".format((df["A"] < 5).sum()))

# percentage
(df["A"] < 5).mean()
print("In the columns A the values that are below 5 represent {}%".format((df["A"] < 5).mean()))

<a id = "trick78"></a>
# Trick 78: Realice un seguimiento de la procedencia de sus datos cuando utilice varias fuentes
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
# generemos algunos datos falsos
df1 = generate_sample_data()
df2 = generate_sample_data()
df3 = generate_sample_data()
# df1.head()
# df2.head()
# df3.head()
df1.to_csv("trick78data1.csv")
df2.to_csv("trick78data2.csv")
df3.to_csv("trick78data3.csv")

# Step 1 generar lista con el nombre del archivo
lf = []
for _,_, files in os.walk("/kaggle/working/"):
    for f in files:
        if "trick78" in f:
            lf.append(f)
            
lf

# Puede usar esto en su máquina local
#desde glob import glob
#files = glob("trick78.csv")

# Step 2: Assing crea una nueva columna llamada nombre de archivo y el valor es archivo
# Aparte de esto, solo estamos concatenando los diferentes marcos de datos
df = pd.concat((pd.read_csv(file).assign(filename = file) for file in lf), ignore_index = True)
df.sample(10)

<a id = "trick77"></a>
# Trick 77: Combine las categorías pequeñas en una sola categoría denominada "Otros" (usando donde)
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
d = {"genre": ["A", "A", "A", "A", "A", "B", "B", "C", "D", "E", "F"]}
df = pd.DataFrame(d)
df["genre"].value_counts()

# Step 1: contar las frecuencias
top_four = df["genre"].value_counts().nlargest(4).index
top_four

# Step 2: actualizar el df
df_updated = df.where(df["genre"].isin(top_four), other = "Other")
df_updated["genre"].value_counts()

<a id = "trick76"></a>
# Trick 76: Filtra en pandas solo las categorías más grandes.
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = pd.read_csv("../input/imdb-data/IMDB-Movie-Data.csv")
df.columns = map(str.lower, list(df.columns)) # convertir encabezados a tipo inferior
df.shape
# seleccione los 3 mejores géneros
top_genre = df["genre"].value_counts().to_frame()[0:3].index

# ahora filtremos el df con el género superior
df_top = df[df["genre"].isin(top_genre)]
df_top
df_top.shape
df_top["genre"].unique()

<a id = "trick75"></a>
# Trick 75: Cuente el número de palabras en una serie de pandas
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = pd.read_csv("../input/imdb-data/IMDB-Movie-Data.csv", usecols=["Title"])
df["Words"] = df["Title"].str.count(" ") + 1
df

<a id = "trick74"></a>
# Trick 74: Webscraping usando read_html() y emparejar el parámetro
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
#Ejecute esto en su computadora
# url = "https://es.wikipedia.org/wiki/Twitter"
# tables = pd.read_html(url)
# len(tables)

# matching_tables = pd.read_html(url, match = "Followers")
# matching_tables[0]

<a id = "trick73"></a>
# Trick 73: Eliminar una columna y almacenarla como una serie separada
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = pd.read_csv("../input/imdb-data/IMDB-Movie-Data.csv")
df.head()

meta = df.pop("Metascore").to_frame()
df.head()
meta.head()

<a id = "trick72"></a>
# Trick 72: Convertir variable continua en categórica (cut and qcut)
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = pd.read_csv("../input/imdb-data/IMDB-Movie-Data.csv")
df.head()

# Usando cortar puede especificar los bordes del contenedor
pd.cut(df["Metascore"], bins = [0, 25, 50, 75, 99]).head()

# Usando qcut puede especificar el número de bins y llenar generar de aproximadamente el mismo tamaño
pd.qcut(df["Metascore"], q = 3).head()

# cut y qcut aceptan el tamaño de la bandeja de etiquetas
pd.qcut(df["Metascore"], q = 4, labels = ["awful", "bad", "average", "good"]).head()

<a id = "trick71"></a>
# Trick 71: Leer datos de un PDF (tabula py)
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
# corre esto en tu computadora
#desde la importación de tabula read_pdf
# df = read_pdf("test.pdf", pages = "all")

<a id = "trick70"></a>
# Trick 70: Imprime la versión actual de pandas y sus dependencias
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
print(pd.__version__)
print(pd.show_versions())

<a id = "trick69"></a>
# Trick 69: Compruebe si 2 series son "similares"
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
d = {"A":[1, 2, 3, 4,], "B":[1.0, 2.0, 3.0, 4.0], "C":[1.00000, 2.00000, 3.00000, 4.000003], "D":[1.0, 2.0, 3.0, 4.0], "E":[4.0, 2.0, 3.0, 1.0]}
df = pd.DataFrame(d)
df

df["A"].equals(df["B"]) # Ellos requieren tipos de datos idénticos
df["B"].equals(df["C"])
df["B"].equals(df["D"])
df["B"].equals(df["E"]) # y otro orden

print(pd.testing.assert_series_equal(df["A"], df["B"], check_names=False, check_dtype=False)) # pasa la afirmación

<a id = "trick68"></a>
# Trick 68: Webscraping usando read_html()
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
# tu debes correr esto en tu computadora
#apple_stocks = pd.read_html("https://finance.yahoo.com/quote/AAPL/history?p=AAPL")
#pd.concat([apple_stocks[0], apple_stocks[1]])

<a id = "trick67"></a>
# Trick 67: Crear nuevas columnas o sobrescriba usando assing y establezca un título para el df
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
print_files()

df = pd.read_csv("/kaggle/input/drinks-by-country/drinksbycountry.csv", usecols=["continent", "beer_servings"])
df.head()

(df.assign(continent = df["continent"].str.title(),
           beer_ounces = df["beer_servings"]*12,#                                     
           beer_galons = lambda df: df["beer_ounces"]/128).query("beer_galons > 30").style.set_caption("Average beer consumption"))

<a id = "trick66"></a>
# Trick 66: Crear un montón de columnas nuevas 
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
d = {"state":["ny", "CA", "Tx", "FI"], "country":["USA", "usa", "UsA", "uSa"], "pop":[1000000, 2000000, 30000, 40000]}
df = pd.DataFrame(d)
df

int_types = ["int64"]
# creating new columns
for col in df.columns:
    ctype = str(df[col].dtype)
    if ctype in int_types:
        df[f'{col}_millions'] = df[col]/1000000
    elif ctype == "object":
        df[f'{col}_new'] = df[col].str.upper()
        # you can also drop the columns
        df.drop(col, inplace = True, axis = "columns")
        
df

<a id = "trick65"></a>
# Trick 65: Seleccionar columnas usando f-strings (new in pandas 3.6+)
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = pd.read_csv("/kaggle/input/drinks-by-country/drinksbycountry.csv")
df

drink = "wine"

# nos permite iteractuar rápidamente sobre columnas
df[f'{drink}_servings'].to_frame()

<a id = "trick64"></a>
# Trick 64: Fijación "SettingWithCopyWarning" cuando creas nuevas columnas
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = pd.DataFrame({"gender":["Male", "Female", "Female", "Male"]})
df

# Recibiendo esta desagradable advertencia
males = df[df["gender"] == "Male"]
males["abbreviation"] = "M"

# Fijación de un errpr
print("Fixing the warning with print")
males = df[df["gender"] == "Male"].copy()
males["abbreviation"] = "M"
males

<a id = "trick63"></a>
# Trick 63: Calcule el recuento corriente con grupos usando cumcount() + 1
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
d = {"salesperson":["Nico", "Carlos", "Juan", "Nico", "Nico", "Juan", "Maria", "Carlos"], "item":["Car", "Truck", "Car", "Truck", "cAr", "Car", "Truck", "Moto"]}
df = pd.DataFrame(d)
df

# Fixing columns
df["salesperson"] = df["salesperson"].str.title()
df["item"] = df["item"].str.title()

df["count_by_person"] = df.groupby("salesperson").cumcount() + 1
df["count_by_item"] = df.groupby("item").cumcount() + 1
df["count_by_both"] = df.groupby(["salesperson","item"]).cumcount() + 1
df

<a id = "trick62"></a>
# Trick 62: Fijando "SettingWithCopyWarning" al cambiar columnas usando loc
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = pd.DataFrame({"gender":["Male", "Female", "Female", "Male"]})
df

# Recibiendo esta desagradable advertencia
df[df["gender"] == "Male"]["gender"] = 1
df[df["gender"] == "Female"]["gender"] = 0


print("Fix using loc")
df.loc[df["gender"] == "Male", "gender"] = 1
df.loc[df["gender"] == "Female", "gender"] = 0
df

<a id = "trick61"></a>
# Trick 61: Leer JSON de la web en un df
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
url = "https://github.com/justmarkham?tab=repositories"

# correr esto en tu computadora
# df = pd.read_json(url)
# df = df[df["fork"] == False]
# df.shape
# df.head()

# lc = ["name", "stargazers_count", "forks_count"]
# df[lc].sort_values("stargazers_count", asending = False).head(10)

<a id = "trick60"></a>
# Trick 60: Creación de totales acumulados con la función cumsum
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
d = {"salesperson":["Nico", "Carlos", "Juan", "Nico", "Nico", "Juan", "Maria", "Carlos"], "item":[10, 120, 130, 200, 300, 550, 12.3, 200]}
df = pd.DataFrame(d)
df

df["running_total"] = df["item"].cumsum()
df["running_total_by_person"] = df.groupby("salesperson")["item"].cumsum()
df

# otras funciones útiles son cummax(), cummin(), cumprod()

<a id = "trick59"></a>
# Trick 59: Combinar output de una agregación con el df original usando una transformación
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
d = {"orderid":[1, 1, 1, 2, 2, 3, 4, 5], "item":[10, 120, 130, 200, 300, 550, 12.3, 200]}
df = pd.DataFrame(d)
df

print("This is the output we want to aggregate to the original df")
df.groupby("orderid")["item"].sum().to_frame()

df["total_items_sold"] = df.groupby("orderid")["item"].transform(sum)
df

<a id = "trick58"></a>
# Trick 58: Utilice encabezados y saltos para deshacerse de datos incorrectos o filas vacías durante la importación
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
# tenemos filas vacías y datos incorrectos
df = pd.read_csv("/kaggle/input/trick58data/trick58data.csv")
df

# importando los datos
df = pd.read_csv("/kaggle/input/trick58data/trick58data.csv", header = 2, skiprows = [3,4])
df

<a id = "trick57"></a>
# Trick 57: Accediendo a los grupos de un objeto groupby (get_group())
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
print_files()

df = pd.read_csv("/kaggle/input/imdb-data/IMDB-Movie-Data.csv")
df

gbdf = df.groupby("Genre")
gbdf.get_group("Horror")

<a id = "trick56"></a>
# Trick 56: Aplicar asignaciones o funciones a todo el df (applymap)
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = pd.DataFrame({"A":["Male", "Female", "Female", "Male"], "B":["x", "y", "z", "A"], "C":["male", "female", "male", "female"], "D":[1, 2, 3, 4]})
df

# primero usemos applymap para convertir y estandarizar el texto
df = df.applymap(lambda x: x.lower() if type(x) == str else x)

mapping = {"male":0, "female":1}

print("PROBLEM: Applies to the whole df but retruns None")
df.applymap(mapping.get)

print("Get the correct result but you have to specify the colums. If you don't want to do this, check the next result")
df[["A", "C"]].applymap(mapping.get)

print("Condtional apply map: if can map --> map else return the same value")
df = df.applymap(lambda x: mapping[x] if x in mapping.keys() else x)
df

<a id = "trick55"></a>
# Trick 55: Filtrar un df con múltiples criterios usando reducir
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = pd.read_csv("/kaggle/input/drinks-by-country/drinksbycountry.csv")
df

print("Classical filter hard to read and mantain.")
df[(df["continent"] == "Europe") & (df["beer_servings"] > 150) & (df["wine_servings"] > 50) & (df["spirit_servings"] < 60)]

print("You can split it across multiple lines to make it more readable. But it's still hard to read.")
df[
    (df["continent"] == "Europe") & 
    (df["beer_servings"] > 150) & 
    (df["wine_servings"] > 50) & 
    (df["spirit_servings"] < 60)
]

print("Solution saving criteria as objects")

cr1 = df["continent"] == "Europe"
cr2 = df["beer_servings"] > 150
cr3 = df["wine_servings"] > 50
cr4 = df["spirit_servings"] < 60

df[cr1 & cr2 & cr3 & cr4]

print("Solution using reduce")
from functools import reduce

# creates our criteria usings lambda
# lambda takes 2 parameters, x and y
# reduce combines them & for every cr in the (cr1, cr2, cr3, cr4)
criteria = reduce(lambda x, y: x & y, (cr1, cr2, cr3, cr4))
df[criteria]


<a id = "trick54"></a>
# Trick 54: Calcula la diferencia entre cada fila y la anterior. (diff())
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = generate_sample_data()
df["A_diff"] = df["A"].diff() # calcular la diferencia entre 2 filas
df["A_diff_pct"] = df["A"].pct_change()*100 # calcula la variación porcentual entre 2 filas

# y un solo estilo
df.style.format({"A_diff_pct":'{:.2f}%'})


<a id = "trick53"></a>
# Trick 53: Arrastrar filas de un df (df.sample())
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = generate_sample_data()

df.sample(frac = 0.5, random_state = 2)
df.sample(frac = 0.5, random_state = 2).reset_index(drop = True) # restablecer el índice después de arrastrar


<a id = "trick52"></a>
# Trick 52: Hacer parcelas con pandas
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = generate_sample_data()

df.plot(kind = "line")
df.plot(kind = "bar")
df.plot(kind = "barh")
df.plot(kind = "hist")
df.plot(kind = "box")
df.plot(kind = "kde")
df.plot(kind = "area")

# the following plots requiere x and y
df.plot(x = "A", y = "B", kind = "scatter")
df.plot(x = "A", y = "B", kind = "hexbin")
df.plot(x = "A", y = "B", kind = "pie") # aquí puede pasar solo, pero necesita agregar subparcelas = True

# otras parcelas están disponibles a través de pd.plotting
# más sobre trazar https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html

<a id = "trick51"></a>
# Trick 51: Concatenar cadenas de 2 columnas
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
print_files()

df = pd.read_csv("/kaggle/input/titanic/train.csv")

# Solución 1: usando str.cat 
df["Name"].str.cat(df["Sex"], sep = ", ").head()

# using + sign
df["Name"] + ", " + df["Sex"].head()

<a id = "trick50"></a>
# Trick 50: Agregación con nombre con varias columnas que pasan tupples (nuevo en pandas 0.25)
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = pd.read_csv("/kaggle/input/titanic/train.csv")

# Grupo típico
print("Problem: MultiIndex")
df.groupby("Pclass").agg({"Age":["mean", "max"], "Survived": "mean"})

#Tenga en cuenta que esto se ha cubierto en 86 y 86 bis.
# Esta es solo una forma más de hacerlo.
print("Named aggregation")
df.groupby("Pclass").agg(avg_age = ("Age", "mean"),
                        max_age = ("Age", "max"), 
                        survival_rate = ("Survived", "mean"))

<a id = "trick49"></a>
# Trick 49: Muestreo con pandas (con reemplazo y pesos)
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
d = {"A": [100, 200, 300, 400, 100], "W":[10, 5, 0, 3, 8]}
df = pd.DataFrame(d)
df

# con reemplazo
df.sample(n = 5, replace = True, random_state = 2)

# sumando pesos
df.sample(n = 5, replace = True, random_state = 2, weights = "W")


<a id = "trick48"></a>
# Trick 48: Parámetros útiles al usar pd.read_csv()
[Go back to the Table of Contents](#table_of_contents)

In [ ]:

df = pd.read_csv("/kaggle/input/drinks-by-country/drinksbycountry.csv")
df.head()
df.dtypes

# Vamos a importar las columnas country y beer_servings, convertirlas en cadenas y float64 respectivamente
# Importe solo las primeras 5 filas e hilo 0 como nans
df = pd.read_csv("/kaggle/input/drinks-by-country/drinksbycountry.csv",
                    usecols=["country", "beer_servings"],
                    dtype={"country":"category", "beer_servings":"float64"},
                    nrows = 5,
                    na_values = 0.0)
df.head()
df.dtypes

# more about read_csv on https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

<a id = "trick47"></a>
# Trick 47: Cree una fila para cada elemento de una lista (explode)
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
d = {"Team":["FC Barcelona", "FC Real Madrid"], 
    "Players":[["Ter Stegen", "Semedo", "Piqué", "Lenglet", "Alba", "Rakitic", "De Jong", "Sergi Roberto", "Messi", "Suárez", "Griezmann"], \
               ["Courtois", "Carvajal", "Varane", "Sergio Ramos", "Mendy", "Kroos", "Valverde", "Casemiro", "Isco", "Benzema", "Bale"]]}

print("Tenga en cuenta que tenemos una lista de jugadores para cada equipo. Generemos una fila para cada jugador..")
df = pd.DataFrame(d)
df

print("Usando explotar para generar nuevas filas para cada jugador.")
df1 = df.explode("Players")
df1

print("Invierta esta operación con groupby y agg")
df["Imploded"] = df1.groupby(df1.index)["Players"].agg(list)
df

<a id = "trick46"></a>
# Trick 46: Almacene NaN en un tipo entero con Int64 (not int64)
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
print("Serie predeterminada")
ser1 = pd.Series([10, 20])
ser1

print("Agreguemos un NaN a una serie int64")
ser1 = pd.Series([10, 20, np.nan])
ser1 # Notice it has been converted to float64

print("Pero si usamos Int64, todo funcionará.")
ser1 = pd.Series([10, 20, np.nan], dtype = "Int64")
ser1

<a id = "trick45"></a>
# Trick 45: Crear filas para valores separados por comas en una celda (Asignar y explorar)
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
d = {"Team":["FC Barcelona", "FC Real Madrid"], 
    "Players":["Ter Stegen, Semedo, Piqué, Lenglet, Alba, Rakitic, De Jong, Sergi Roberto, Messi, Suárez, Griezmann",
               "Courtois, Carvajal, Varane, Sergio Ramos, Mendy, Kroos, Valverde, Casemiro, Isco, Benzema, Bale"]}

print("Tenga en cuenta que tenemos una lista de jugadores para cada equipo separados por comas. Generemos una fila para cada jugador..")
df = pd.DataFrame(d)
df

print("Observe que nos hemos convertido a algo similar visto en el ejemplo 47..")
df.assign(Players = df["Players"].str.split(","))

print("Ahora agregue explotar y listo.")
df.assign(Players = df["Players"].str.split(",")).explode("Players")

<a id = "trick44"></a>
# Trick 44: Use una variable local dentro de una consulta en pandas (usando @)
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = generate_sample_data()
df

# crear una media de variable local
mean = df["A"].mean()

# ahora usemos dentro de una consulta de pandas usando @
df.query("A > @mean")


<a id = "trick43"></a>
# Trick 43: Crea una fila para cada elemento en una lista (explotar) !!! Truco 47 duplicado !!!
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
# Parece que este truco está duplicado, pasa al siguiente
# Decidí quedarme, así que en el futuro no habrá confusión si consulta el material original
# y este kernel
d = {"Team":["FC Barcelona", "FC Real Madrid"], 
    "Players":[["Ter Stegen", "Semedo", "Piqué", "Lenglet", "Alba", "Rakitic", "De Jong", "Sergi Roberto", "Messi", "Suárez", "Griezmann"], \
               ["Courtois", "Carvajal", "Varane", "Sergio Ramos", "Mendy", "Kroos", "Valverde", "Casemiro", "Isco", "Benzema", "Bale"]]}

print("Tenga en cuenta que tenemos una lista de jugadores para cada equipo. Generemos una fila para cada jugador..")
df = pd.DataFrame(d)
df

print("Usando explotar para generar nuevas filas para cada jugador.")
df1 = df.explode("Players")
df1

print("Invierta esta operación con groupby y agg")
df["Imploded"] = df1.groupby(df1.index)["Players"].agg(list)
df

<a id = "trick42"></a>
# Trick 42: Nueva función de agregación --> last()
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
d = {"patient":[1, 2, 3, 1, 1, 2], "visit":[2015, 2016, 2014, 2016, 2017, 2020]}
df = pd.DataFrame(d)
df.sort_values("visit")

print("Hagamos la última visita para cada paciente.")
df.groupby("patient")["visit"].last().to_frame()

<a id = "trick41"></a>
# Trick 41: Ordenar Categorias (from pandas.api.types import CategoricalDtypee)
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
import pandas as pd
from pandas.api.types import CategoricalDtype
d = {"ID":[100, 101, 102, 103], "quality":["bad", "very good", "good", "excellent"]}
df = pd.DataFrame(d)
df

print("Creemos nuestro propio orden categórico.")
cat_type = CategoricalDtype(["bad", "good", "very good", "excellent"], ordered = True)
df["quality"] = df["quality"].astype(cat_type)
df

print("Ahora podemos usar la ordenación lógica.")
df = df.sort_values("quality", ascending = True)
df

print("También podemos filtrar esto como si fueran números. ASOMBROSO.")
df[df["quality"] > "bad"]

<a id = "trick40"></a>
# Trick 40: Estilo que df rápido con hide_index() and set_caption()
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = generate_sample_data()
print("Original df")
df

df.style.hide_index().set_caption("Estilo df sin índice y título")

<a id = "trick39"></a>
# Trick 39: Una codificación en caliente (get_dummies())
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = pd.read_csv("/kaggle/input/titanic/train.csv", usecols = [2, 4, 5, 11], nrows = 10)
df

pd.get_dummies(df) # Observe que podemos eliminar una columna de cada una ya que esta información está contenida en las demás

pd.get_dummies(df, drop_first=True)

<a id = "trick38"></a>
# Trick 38: Pandas datetime (todos los ejemplos)
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = generate_sample_data_datetime().reset_index()
df = df.sample(500)
df["Year"] = df["index"].dt.year
df["Month"] = df["index"].dt.month
df["Day"] = df["index"].dt.day
df["Hour"] = df["index"].dt.hour
df["Minute"] = df["index"].dt.minute
df["Second"] = df["index"].dt.second
df["Nanosecond"] = df["index"].dt.nanosecond
df["Date"] = df["index"].dt.date
df["Time"] = df["index"].dt.time
df["Time_Time_Zone"] = df["index"].dt.timetz
df["Day_Of_Year"] = df["index"].dt.dayofyear
df["Week_Of_Year"] = df["index"].dt.weekofyear
df["Week"] = df["index"].dt.week
df["Day_Of_week"] = df["index"].dt.dayofweek
df["Week_Day"] = df["index"].dt.weekday
df["Week_Day_Name"] = df["index"].dt.weekday_name
df["Quarter"] = df["index"].dt.quarter
df["Days_In_Month"] = df["index"].dt.days_in_month
df["Is_Month_Start"] = df["index"].dt.is_month_start
df["Is_Month_End"] = df["index"].dt.is_month_end
df["Is_Quarter_Start"] = df["index"].dt.is_quarter_start
df["Is_Quarter_End"] = df["index"].dt.is_quarter_end
df["Is_Leap_Year"] = df["index"].dt.is_leap_year
df

<a id = "trick37"></a>
# Trick 37: Pandas cortando loc e iloc (6 ejemplos)
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = generate_sample_data()
df

# usando loc --> etiquetas
df.loc[0, "A"]

# usando iloc --> posición
df.iloc[0, 0]

# mezcla de etiquetas y posición con loc
df.loc[0, df.columns[0]]

# mezcla de etiquetas y posición con loc
df.loc[df.index[0], "A"]

# mezcla de etiquetas y posición con iloc
df.iloc[0, df.columns.get_loc("A")]

# mezcla de etiquetas y posición con iloc
df.iloc[df.index.get_loc(0), 0]

<a id = "trick36"></a>
# Trick 36: Convertir de UTC a otra zona horaria
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
s = pd.Series(range(1552194000, 1552212001, 3600))
s = pd.to_datetime(s, unit = "s")
s

# Establecer zona horaria (UTC)
s = s.dt.tz_localize("UTC")
s

# zona horaria especifica (Chicago)
s = s.dt.tz_convert("America/Chicago")
s

<a id = "trick35"></a>
# Trick 35: Consultar una columna que tenga espacios en el nombre (usando comillas invertidas)
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
d = {"colum_without_space":np.array([1, 2, 3, 4, 5, 6]), "column with space":np.array([1, 2, 3, 4, 5, 6])*2}
df = pd.DataFrame(d)
df

print("Consultar una columna sin espacio")
df.query("colum_without_space > 4")
print("Consultar una columna con espacio usando comillas invertidas ``")
print("Esto es una tilde ``")
df.query("`column with space` > 8")

<a id = "trick34"></a>
# Trick 34: Explore un conjunto de datos con creación de perfiles
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
import pandas_profiling

df = generate_sample_data()

df

print("Generación de informes con perfiles de pandas")
df.profile_report()


<a id = "trick33"></a>
# Trick 33: Opciones de visualización de pandas
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
# use pd.describe_option() to see all
# max_rows
# max_columns
# max_colwidth
# precision
# date_dayfirst
# date_yearfirst

df = generate_sample_data_datetime()[:10].reset_index()
df["sales"] = df["sales"].astype("float")
df

pd.set_option("display.max_rows",5)
pd.set_option("display.max_columns",3)
pd.set_option('display.width', 1000)
pd.set_option('display.date_dayfirst', True)
pd.describe_option()

pd.reset_option('^display.', silent=True) # restore to default
#pd.reset_option('display.width') # restore one by one

<a id = "trick32"></a>
# Trick 32: Filtrar un df con consulta y evitar variables intermedias
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = generate_sample_data()[:10]
df["A"] = pd.Series(["APP", "GOO", "APP", "GOO", "MIC", "MIC", "APP", "GOO", "MIC", "APP"])
df.rename(columns = {"A":"stock"}, inplace = True)
print("Original df")
df

print("Filtrar datos usando variables intermedias")
temp = df.groupby("stock").mean()
temp 

fv = temp["B"].sort_values(ascending = False)[1] # filter by the second greates. This way every time we generate sample data we will have a result
temp[temp["B"] < fv]

print("Filtrar mediante consulta")
df.groupby("stock").mean().query("B < {}".format(fv))
df.groupby("stock").mean().query("B < @fv")
df.groupby("stock").mean().query("B < 10")

<a id = "trick31"></a>
# Trick 31: Ver todas las columnas de un gran df
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
pd.reset_option('^display.', silent=True) # restore to default

df = generate_sample_data()
df1 = df.copy(deep = True)
df = df.append(df1)

print("Imagina que tenemos un gran df donde podemos ver todas las columnas. ...")
df.T.head() # we are trasposing JUST TO CREATE A GIANT DF

# Solution 1
print("Solución 1 pd.set_option display.max_columns")
pd.set_option("display.max_columns", None)
df.T.head()
pd.reset_option('^display.', silent=True) # restore to default

# Solution 2
print("Otra solución inteligente con Traspose")
df.T.head().T

<a id = "trick30"></a>
# Trick 30: Los pandas se fusionan -> vea de dónde vienen las columnas (indicador = Verdadero)
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = generate_sample_data()
df1 = df.copy(deep = True)
df1 = df1.drop([0, 1, 2], axis = "rows") 
df.head()
df1.head()

pd.merge(df, df1, how = "left", indicator = True)


<a id = "trick29"></a>
# Trick 29: Acceda a numpy dentro de pandas (sin importar numpy como np)
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
# Pandas se basa en numpy, por lo que podemos acceder a todas las funciones de numpy de pandas
pd.np.random.rand(2, 3)
pd.np.nan

<a id = "trick28"></a>
# Trick 28: Agregando por múltiples columnas (usando agg)
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = pd.read_csv("/kaggle/input/drinks-by-country/drinksbycountry.csv")
print("Original df")
df

print("Porciones de cerveza por continente")
df.groupby("continent")["beer_servings"].mean()

print("Usando agg para pasar múltiples funciones")
df.groupby("continent")["beer_servings"].agg(["mean", "count"])

print("Usar describir sobre un objeto groupby")
df.groupby("continent")["beer_servings"].describe()

<a id = "trick27"></a>
# Trick 27: Agregación sobre series temporales (remuestreo)
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = generate_sample_data_datetime()

print("Original df")
df
print("Volvamos a muestrear / agrupar por mes")
df.resample("M")["sales"].sum()

print("Volvamos a muestrear / agrupar por día")
df.resample("D")["sales"].sum()

<a id = "trick26"></a>
# Trick 26: Formatear diferentes columnas de un df (usando diccionarios)
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = generate_sample_data_datetime().reset_index()[:10]
df.rename(columns = {"index":"time"}, inplace = True)
df["sales_100"] = df["sales"]*100
print("Original df")
df.head()

# declare a formatting dict: individual for each column
fd = {"time":"{:%d/%m/%y}", "sales":"${:.2f}", "customers":"{:,}"}
df.style.format(fd)
df

# add some more formattin
(df.style.format(fd)
 .hide_index()
 .highlight_min("sales", color ="red")
 .highlight_max("sales", color ="green")
 .background_gradient(subset = "sales_100", cmap ="Blues")
 .bar("customers", color = "lightblue", align = "zero")
 .set_caption("A df with different stylings")
)

<a id = "trick25"></a>
# Trick 25: 3 formas de cambiar el nombre de las columnas
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = generate_sample_data()
df.head(2)

# Solución 1
df.rename({"A":"col_1", "B":"col_2"}, axis = "columns", inplace = True)
df.head(2)

# Solución 2
df.columns = ["col1", "col2", "col3", "col4","col5", "col6", "col7"] # la lista debe ser igual al número de columnas
df.head(2)

# Solución 3
df.columns = df.columns.str.title() # aplicar cualquier método de cadena a los nombres de las columnas
df.head(2)

<a id = "trick24"></a>
# Trick 24: Copie datos de Excel a pandas rápidamente (read_clipboard())
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
# Deberas checar en tu computadora
# Útil para importar rápidamente
# Step 1: copiar una tabla de la hoja de Excel usando ctrl + c (to the clipboard)
# Step 2: ejecutar este comando
# df = pd.read_clipboard()

<a id = "trick23"></a>
# Trick 23: Complete los valores faltantes en los datos de la serie temporal (interpolate ())
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
d = {"col1":[100, 120 ,140, np.nan, 160], "col2":[9, 10, np.nan, 7.5, 6.5]}
df = pd.DataFrame(d)
df.index = pd.util.testing.makeDateIndex()[0:5]
print("Original df")
df
print("DataFrame despues")
df.interpolate()

<a id = "trick22"></a>
# Trick 22: Crear DataFrames para probar
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
print("Contiene valores aleatorios")
df1 = pd.util.testing.makeDataFrame() 
df1
print("Contiene valores perdidos")
df2 = pd.util.testing.makeMissingDataframe() 
print("Contiene valores de fecha y hora")
df3 = pd.util.testing.makeTimeDataFrame() 
df3
print("Contiene valores mixtos")
df4 = pd.util.testing.makeMixedDataFrame() 
df4

<a id = "trick21"></a>
# Trick 21: dividir una columna de cadena en varias columnas
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
d = {"name":["John Artur Doe", "Jane Ann Smith", "Nico P"], "location":["Los Angeles, CA", "Washington, DC", "Barcelona, Spain"]}
df = pd.DataFrame(d)
df

df[["first", "middle", "last"]] = df["name"].str.split(" ", expand = True)
df["city"] = df["location"].str.split(",", expand = True)[0]
df

<a id = "trick20"></a>
# Trick 20: Crear columnas de fecha y hora a partir de varias columnas
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
d = {"day":[1, 2, 10 ,25, 12], "month":[1, 2, 4, 5, 6], "year":[2000, 2001, 2010, 2015, 2020]}
df = pd.DataFrame(d)
df["date"] = pd.to_datetime(df[["day", "month", "year"]])
df
df.dtypes

<a id = "trick19"></a>
# Trick 19: Muestra el uso de memoria de un df y cada columna
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = generate_sample_data_datetime().reset_index()
df.columns = ["date", "sales", "customers"]
df

print("Muestra el uso global de la memoria del df")
df.info(memory_usage = "deep")
print()
print("Muestra el uso de memoria de cada columna.")
df.memory_usage(deep = True)

<a id = "trick18"></a>
# Trick 18: Leer y escribir en un archivo comprimido (csv.zip)
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = generate_sample_data()
df.head()

print("Escribir datos en un archivo csv.zip")
df.to_csv("trick18data.csv.zip")

print("Deleting df")
del df

print("Importación de datos de un archivo csv.zip")
df = pd.read_csv("/kaggle/working/trick18data.csv.zip", index_col=0)
df.head()



<a id = "trick17"></a>
# Trick 17: Seleccione varias filas y columnas con loc
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = generate_sample_data()
print("Original df")
df

print("Usando una rebanada (inclusive)")
df.loc[0:4, "A":"E"]

print("Usando una lista")
df.loc[[0,4], ["A","E"]]

print("Usando una condición")
df.loc[df["A"] > 10, ["A","E"]]

<a id = "trick16"></a>
# Trick 16: Convertir valores continuos en categóricos (cortar ())
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = generate_sample_data()
df["A"] = df["A"] + 5
df.rename(columns = {"A":"age"}, inplace = True)
df.sample(5)

df["age_groups"] = pd.cut(df["age"], bins = [0, 18, 65, 99], labels = ["kids", "adult", "elderly"])
df

<a id = "trick15"></a>
# Trick 15: Cambiar la forma de un MultiIndex df (unstack())
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = pd.read_csv("/kaggle/input/titanic/train.csv")
print("Original df")
df.head()

print("Groupby y crear un MultiIndex df")
print("Tenga en cuenta que tenemos un df con MultiIndex (Sex y Pclass)")
df.groupby(["Sex", "Pclass"])["Survived"].mean().to_frame()

print("Remodelar usando desapilar")
print("Ahora podemos interactuar con él como con un df normal.")
df.groupby(["Sex", "Pclass"])["Survived"].mean().unstack()

<a id = "trick14"></a>
# Trick 14: Creando juguete df (3 métodos)
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
# Método 1: de un dictado
pd.DataFrame({"A":[10 ,20], "B":[30, 40]})

# Método 2: usando numpy
pd.DataFrame(np.random.rand(2, 3), columns = list("ABC"))

# Método 3: usar funcionalidades integradas de pandas
pd.util.testing.makeMixedDataFrame()

<a id = "trick13"></a>
# Trick 13: Evita la serie de listas TRAP
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
d = {"A":[1, 2, 3], "B":[[10, 20], [40, 50], [60, 70]]}
df = pd.DataFrame(d)
print("Observe que la columna B tiene listas de valores")
df
print("Conviértelo a serie normal")
df_ = df["B"].apply(pd.Series)
df_

print("unirse the 2 df")
pd.merge(df, df_, left_index = True, right_index = True)


<a id = "trick12"></a>
# Trick 12: Fusionar conjuntos de datos y verificar la unicidad
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = generate_sample_data()[:10]
df1 = df.copy(deep = True)
df = df.drop([0, 1, 2])
df1 = df1.drop([8, 9])
df
df1

df_one_to_one = pd.merge(df, df1, validate = "one_to_one")
df_one_to_one

df_one_to_many = pd.merge(df, df1, validate = "one_to_many")
df_one_to_many

df_many_to_one = pd.merge(df, df1, validate = "many_to_one")
df_many_to_one


<a id = "trick11"></a>
# Trick 11: Cambiar el nombre de todas las columnas con el mismo patrón
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
print_files()
df = pd.read_csv("/kaggle/input/titanic/train.csv")
df.columns = ["Passenger ID", "Survived", "Pclass", "Name         ", "Sex", "Age", "Sib SP", "Parch", "Ticket", "Fare", "Cabin", "Embarked"] # creating column names for the example
df
df1 = df.copy(deep = True)

print("Reemplazar todos los espacios con undescore y convertir a menor")
print("Observe que la columna SP de pasajero y hermano ahora tiene un guión bajo")
df.columns = df.columns.str.replace(" ", "_").str.lower()
df.head()

print("Quite la parte blanca final (al final) y conviértala en menor")
print("Observe que la columna SP de pasajero y hermano ahora tiene un guión bajo")
df1.columns = df1.columns.str.lower().str.rstrip()
df1.head()

<a id = "trick10"></a>
# Trick 10: Verifica la igualdad de 2 series
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = generate_sample_data()[["A", "B"]][:5]
df["A"] = pd.Series([15, 15, 18, np.nan, 12])
df["B"] = pd.Series([15, 15, 18, np.nan, 12])
df

print("No usar ==, no maneja NaN correctamente")
print("Observe que el elemento 4 de cada lista es np.nan pero == aún devuelve False")
df["A"] == df["B"]

print("Usando iguales. Ahora obtenemos True, por lo que las 2 series son iguales")
df["A"].equals(df["B"])

print("Igual también funciona para df")
df1 = df.copy(deep = True)
df.equals(df1)

print("== de df tiene el mismo problema que para la serie")
df == df1

<a id = "trick9"></a>
# Trick 9: ¡Reduzca el uso de memoria de un df al importar! ¡¡¡Truco 83 duplicado !!!
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
print_files()

df = pd.read_csv("/kaggle/input/imdb-data/IMDB-Movie-Data.csv", \
                 usecols = ["Title", "Genre", "Year", "Metascore", "Revenue (Millions)"])
df.dtypes
df.memory_usage(deep = True)

print("Importando solo unas pocas columnas y convirtiendo al tipo de archivo adecuado")
df = pd.read_csv("/kaggle/input/imdb-data/IMDB-Movie-Data.csv", \
                 usecols = ["Title", "Genre", "Year", "Metascore", "Revenue (Millions)"], \
                dtype = {"Genre":"category", "Metascore":"Int64", "Year":"int8"})
df.dtypes
df.memory_usage(deep = True) #observe cómo el género y el año consumen ahora menos memoria

<a id = "trick8"></a>
# Trick 8: Usando glob para generar un df a partir de múltiples archivos !!! truco duplicado 78 !!!
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
# let's generate some fake data
df1 = generate_sample_data()
df2 = generate_sample_data()
df3 = generate_sample_data()
# df1.head()
# df2.head()
# df3.head()
df1.to_csv("trick8data1.csv", index = False)
df2.to_csv("trick8data2.csv", index = False)
df3.to_csv("trick8data3.csv", index = False)

# Step 1 generate list with the file name
lf = []
for _,_, files in os.walk("/kaggle/working/"):
    for f in files:
        if "trick8data" in f:
            lf.append(f)
            
lf

# You can use this on your local machine
#from glob import glob
#files = glob("trick8.csv")

# Step 2: we do the same as in trick 78 except we don't create a new column of the rows origin (file they came from)
df = pd.concat((pd.read_csv(file) for file in lf), ignore_index = True)
df

<a id = "trick7"></a>
# Trick 7: Manejo de valores perdidos (NaN)
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = pd.util.testing.makeMissingDataframe().reset_index() # contains missing values
df.rename(columns = {"index":"A"})
df1 = df.copy(deep = True)
df

print("Calcule el% de valores perdidos en cada fila")
df.isna().mean() # calcular el% de valores perdidos en cada fila
print("Descartando cualquier columna que tenga valores perdidos. Solo quedará la columna A")
df.dropna(axis = "columns") # eliminar cualquier columna que tenga valores perdidos
print("Eliminar las filas que tienen valores perdidos.")
df1.dropna(axis = "rows") # eliminar cualquier fila que tenga valores perdidos
print("Columna descartada donde los valores faltantes están por encima de un umbral")
df.dropna(thresh = len(df)*0.95, axis = "columns") # eliminar cualquier fila que tenga valores perdidos

<a id = "trick6"></a>
# Trick 6: Dividir un df en 2 subconjuntos aleatorios
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = generate_sample_data()
df_1 = df.sample(frac = 0.7)
df_2 = df.drop(df_1.index) # solo funciona si el índice df es único

df.shape
df_1.shape
df_2.shape

<a id = "trick5"></a>
# Trick 5: Convertir números almacenados como cadenas (coerce)
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
d = {"col1":["1", "2", "3", "stuff"], "col2":["1", "2", "3", "4"]}
df = pd.DataFrame(d)
df.astype({"col2":"int"}) 

print("Tenga en cuenta que ahora las cosas se convirtieron a NaN")
df.apply(pd.to_numeric, errors = "coerce")

<a id = "trick4"></a>
# Trick 4: Seleccionar columnas por tipo de letra
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = generate_sample_data_datetime()[:10].reset_index()
df["string_col"] = list("ABCDEABCDE")
df["sales"] = df["sales"].astype("float")
print("Original df")
df

print("Seleccionar columnas numéricas")
df.select_dtypes(include = "number")

print("Seleccionar columnas de cadena")
df.select_dtypes(include = "object")

print("Seleccionar columna de fecha y horas")
df.select_dtypes(include = ["datetime", "timedelta"])

print("Seleccionar varios")
df.select_dtypes(include = ["number", "object", "datetime", "timedelta"])

print("Seleccione pasando los tipos que necesita")
df.select_dtypes(include = ["int8", "int16", "int32", "int64", "float"])

<a id = "trick3"></a>
# Trick 3: Filtrar un df por múltiples condiciones (inverso)
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = generate_sample_data()[:5]
df["A"] = [1, 2, 3, 4, 5]

print("Filtrar usando múltiples |")
df[(df["A"] == 1) | (df["A"] == 3)]

print("Filtrar usando isin")
df[df["A"].isin([1, 3])]

print("Invert using ~ (ctrl + alt + 4)")
df[~df["A"].isin([1, 3])]

<a id = "trick2"></a>
# Trick 2: Orden inverso de un df
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = generate_sample_data()[:5]
df

print("Orden de columna inverso")
df.loc[:, ::-1]

print("Orden de filas inverso")
df.loc[::-1]

print("Invertir el orden de las filas y restablecer el índice")
df.loc[::-1].reset_index(drop = True)

<a id = "trick1"></a>
# Trick 1: Agregue un prefijo o sufijo a todas las columnas
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df = generate_sample_data()[:5]
print("Original df")
df

print("Add prefix")
df.add_prefix("1_")

print("Add suffix")
df.add_suffix("_Z")


# El fin
# Muchas gracias. Si hiciste hasta el final, habrás aprendido muchos pandas